In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
path_dict = {
    "read_air_reserve" : "/media/andraa/10160545101605452/kaggle/restaurant_forecast/data/raw/air_reserve.csv",
    "read_air_store_info" : "/media/andraa/10160545101605452/kaggle/restaurant_forecast/data/raw/air_store_info.csv",
    "read_air_visit_data" : "/media/andraa/10160545101605452/kaggle/restaurant_forecast/data/raw/air_visit_data.csv",
    "read_date_info" : "/media/andraa/10160545101605452/kaggle/restaurant_forecast/data/raw/date_info.csv",
    "read_hpg_reserve" : "/media/andraa/10160545101605452/kaggle/restaurant_forecast/data/raw/hpg_reserve.csv",
    "read_hpg_store_info" : "/media/andraa/10160545101605452/kaggle/restaurant_forecast/data/raw/hpg_store_info.csv",
    "read_sample_submission": "/media/andraa/10160545101605452/kaggle/restaurant_forecast/data/raw/sample_submission.csv",
    "read_store_id_relation": "/media/andraa/10160545101605452/kaggle/restaurant_forecast/data/raw/store_id_relation.csv"
    
}

# 0.0 Load the data

In [19]:
air_visit_data_df = pd.read_csv(path_dict["read_air_visit_data"])
date_info_df = pd.read_csv(path_dict["read_date_info"])
sample_submission_df = pd.read_csv(path_dict["read_sample_submission"])


In [20]:
# extract the id from the id_data concat

# 0.1 modify sample submission file

In [21]:
sample_submission_df["air_store_id"] = sample_submission_df["id"].apply(lambda x: x[:20])

In [22]:
sample_submission_df["visit_date"] = sample_submission_df["id"].apply(lambda x: x[21:])

# 1.0 extract date related info

In [23]:
# merge with the data containing info on holiday, dya of the week
def extract_data_features(air_visit_data_df, date_info_df):
    
    air_visit_data_df = pd.merge(air_visit_data_df, date_info_df, left_on = "visit_date", right_on = "calendar_date", how = "left")
    air_visit_data_df.drop("calendar_date", axis = 1, inplace=True)

    # transform visit_date to datetime
    air_visit_data_df["visit_date"] = pd.to_datetime(air_visit_data_df["visit_date"])

    # create variable year
    air_visit_data_df["year"] = air_visit_data_df["visit_date"].apply(lambda x: x.year)
    # create variable month
    air_visit_data_df["month"] = air_visit_data_df["visit_date"].apply(lambda x: x.month)

    # apply one hot encoding
    air_visit_data_df["month"] = air_visit_data_df["month"].astype(str)
    air_visit_data_df["year"] = air_visit_data_df["year"].astype(str)

    ohe_day_of_week = pd.get_dummies(air_visit_data_df[["day_of_week"]])
    ohe_year = pd.get_dummies(air_visit_data_df[["year"]])
    ohe_month = pd.get_dummies(air_visit_data_df[["month"]])
    '''
    print(ohe_day_of_week.head())
 
    print(ohe_year.head())
 
    print(ohe_month.head())
    '''
    air_visit_data_df = pd.concat([air_visit_data_df, ohe_day_of_week], axis = 1)
    air_visit_data_df = pd.concat([air_visit_data_df, ohe_year], axis = 1)
    air_visit_data_df = pd.concat([air_visit_data_df, ohe_month], axis = 1)
    
    return air_visit_data_df

In [24]:
air_visit_data_df = extract_data_features(air_visit_data_df, date_info_df)
sample_submission_df = extract_data_features(sample_submission_df, date_info_df)

In [25]:
sample_submission_df.head()

,id,visitors,air_store_id,visit_date,day_of_week,holiday_flg,year,month,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,year_2017,month_4,month_5
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23,Sunday,0,2017,4,0,0,0,1,0,0,0,1,1,0
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24,Monday,0,2017,4,0,1,0,0,0,0,0,1,1,0
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25,Tuesday,0,2017,4,0,0,0,0,0,1,0,1,1,0
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26,Wednesday,0,2017,4,0,0,0,0,0,0,1,1,1,0
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27,Thursday,0,2017,4,0,0,0,0,1,0,0,1,1,0


In [26]:
air_visit_data_df.head()

,air_store_id,visit_date,visitors,day_of_week,holiday_flg,year,month,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,...,month_11,month_12,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9
0,air_ba937bf13d40fb24,2016-01-13,25,Wednesday,0,2016,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,air_ba937bf13d40fb24,2016-01-14,32,Thursday,0,2016,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,air_ba937bf13d40fb24,2016-01-15,29,Friday,0,2016,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,air_ba937bf13d40fb24,2016-01-16,22,Saturday,0,2016,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,air_ba937bf13d40fb24,2016-01-18,6,Monday,0,2016,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
list(air_visit_data_df.columns)

['air_store_id',
 'visit_date',
 'visitors',
 'day_of_week',
 'holiday_flg',
 'year',
 'month',
 'day_of_week_Friday',
 'day_of_week_Monday',
 'day_of_week_Saturday',
 'day_of_week_Sunday',
 'day_of_week_Thursday',
 'day_of_week_Tuesday',
 'day_of_week_Wednesday',
 'year_2016',
 'year_2017',
 'month_1',
 'month_10',
 'month_11',
 'month_12',
 'month_2',
 'month_3',
 'month_4',
 'month_5',
 'month_6',
 'month_7',
 'month_8',
 'month_9']

# 2.0 dumb model

In [27]:
from sklearn import ensemble, neighbors, linear_model, metrics, preprocessing

In [44]:
col = [ 
 'day_of_week_Friday',
 'day_of_week_Monday',
 'day_of_week_Saturday',
 'day_of_week_Sunday',
 'day_of_week_Thursday',
 'day_of_week_Tuesday',
 'day_of_week_Wednesday',
 'year_2016',
 'year_2017',
 'month_1',
 'month_10',
 'month_11',
 'month_12',
 'month_2',
 'month_3',
 'month_4',
 'month_5',
 'month_6',
 'month_7',
 'month_8',
 'month_9']

In [40]:
# add columns that are missing
for column in [ 
 'year_2016',
 'month_6',
 'month_7',
 'month_8',
 'month_9',
 'month_1',
 'month_10',
 'month_11',
 'month_12',
 'month_2',
 'month_3']:
    sample_submission_df[column] = 0

In [34]:
# define models
etc = ensemble.ExtraTreesRegressor(n_estimators=225, max_depth=5, n_jobs=-1, random_state=3)
knn = neighbors.KNeighborsRegressor(n_jobs=-1, n_neighbors=4)


In [35]:
# fit models
etc.fit(air_visit_data_df[col], air_visit_data_df['visitors'].values)
knn.fit(air_visit_data_df[col], air_visit_data_df['visitors'].values)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=-1, n_neighbors=4, p=2,
          weights='uniform')

In [45]:
etc.predict(sample_submission_df[col])

array([ 23.66518505,  17.83419804,  17.83419804, ...,  17.83419804,
        17.83419804,  17.83419804])

In [46]:
sample_submission_df['visitors'] = (etc.predict(sample_submission_df[col]) / 2) +(knn.predict(sample_submission_df[col]) / 2)

In [49]:
sample_submission_df[["id", "visitors"]].to_csv("dumb_submission.csv",index = False)